# How To Compute the Integrals for the Free Energy

This notebook shows how to compute the required energy and gradient functions. That will be used for the MeanFieldVarGP algorithm.

In [ ]:
import os

import sys

import dill

import numpy as np

import sympy as sym

from pathlib import Path

from sympy.stats import E, Normal

from sympy.utilities.iterables import flatten

from copy import deepcopy

sys.path.insert(0, os.path.abspath('../../src/'))

from numerical.symbolics import LagrangePolynomial

In [ ]:
# Help make the output easier to read.
sym.init_printing(use_unicode=True)

# Objects referred to in the global dictionary
# are recursively traced and serialized.
dill.settings['recurse'] = True

## Dynamical System equations

To generate the equations we use the following settings.

In [ ]:
# |------------------|
# | Double Well (1D) |
# |------------------|
# 
# D = 1
# 
# p = sym.Matrix([sym.symbols("theta", real=True)])
# 
# fx = sym.Matrix((4 * xt) * (p - xt**2), real=True)
# 

# |-------------------------|
# | Ornstein-Uhlenbeck (1D) |
# |-------------------------|
# 
# D = 1
# 
# p = sym.Matrix([sym.symbols("theta mu", real=True)])
# 
# fx = sym.Matrix([p[0] * (p[1] - xt[0])], real=True)
# 

# |---------------|
# | Lorenz63 (3D) |
# |---------------|
# 
# D = 3
# 
# p = sym.Matrix([sym.symbols("sigma rho beta", real=True)])
#
# fx = sym.Matrix([p[0]*(xt[1] - xt[0]),
#                 xt[0]*(p[1] - xt[2]) - xt[1],
#                 xt[0]*xt[1] - p[2]*xt[2]], real=True);
# 

# |---------------|
# | Lorenz96 (nD) |
# |---------------|
# 
# D = 4
# 
# p = sym.Matrix([sym.symbols("theta", real=True)])
# 
# fx = sym.Matrix([(xt[1] - xt[3]) * xt[2] - xt[0] + p[0]], real=True)
# 

## First define the main variables.

In [ ]:
# State vector dimensions.
D = 1

# Mean and variance functions: m(t), s(t).
mt = sym.symbols(f"m:{D+1}(t)", real=True)
st = sym.symbols(f"s:{D+1}(t)", real=True, positive=True)

# Random (state) vector: x(t).
xt = sym.Matrix([Normal(f"x{i}(t)", mt[i], sym.sqrt(st[i])) for i in range(D)])

# System noise (diffusion) coefficient vector: sig := sigma^2.
sig = sym.symbols(f"Sig:{D}", real=True, positive=True)

# Model (drift) parameters vector.
p = sym.Matrix([sym.symbols("theta", real=True)])

# Drift function.
fx = sym.Matrix((4 * xt) * (p - xt**2), real=True)

# Define the limits of integration as symbolic variables.
ti, tj = sym.symbols("ti, tj", real=True, positive=True)

# Number of systems dimensions.
# NOTE: this can be different from
#       the state vector dimensions!
dim_D = len(fx)

# Pretty printing.
sym.pprint(fx)

## Compute the mean/variance Lagrange functions.

In [ ]:
# Store the mean and variance functions.
Mfun = []
Sfun = []

# Store the time derivative of the mean and variance functions.
dMt = []
dSt = []

# Store the mean and variance points.
Mpar = []
Spar = []

# Create one polynomial for each state vector dimension.
for i in range(D):
    
    # Create the mean polynomials.
    mt_i, t, mk_i, t_h = LagrangePolynomial(letter=f"d{i}m", order=3, fp="h")
    
    # Replace time points with ti, tj.
    mt_i = mt_i.replace(t_h[0], ti).replace(t_h[3], tj)
    
    # Get the time derivative functions.
    dMt.append(mt_i.diff(t))
    
    # Store the functions.
    Mfun.append(mt_i)
    
    # Store the Lagrange coefficients.
    Mpar.append(mk_i)
    
    # Create the variance polynomials.
    st_i, t, sk_i, t_c = LagrangePolynomial(letter=f"d{i}s", order=2, fp="c")
    
    # Replace time points with ti, tj.
    st_i = st_i.replace(t_c[0], ti).replace(t_c[2], tj)
    
    # Get the time derivative functions.
    dSt.append(st_i.diff(t))
    
    # Store the functions.
    Sfun.append(st_i)
    
    # Store the Lagrange coefficients.
    Spar.append(sk_i)
    
# _end_for_

print("Done!")

## Compute the expectations.

In [ ]:
# E[f(x)]
Efx = E(fx).simplify(); Efx

In [ ]:
# E[f(x)^2]
Efx2 = E(fx.multiply_elementwise(fx)).simplify(); Efx2

In [ ]:
# Jacobian: dfx/dx
dfxdx = fx.jacobian(xt); dfxdx

In [ ]:
# E[dfx/dx]
Edfxdx = E(dfxdx).diagonal().simplify(); Edfxdx

## Replace m(t), s(t) with their polynomial equivalents.

In [ ]:
# Replace the mean functions.
for j, mf_j in enumerate(Mfun):
    Efx = Efx.replace(mt[j], mf_j)
    
    Efx2 = Efx2.replace(mt[j], mf_j)
        
    Edfxdx = Edfxdx.replace(mt[j], mf_j)
# _end_for_

# Replace the variance functions.
for j, sf_j in enumerate(Sfun):
    Efx = Efx.replace(st[j], sf_j)
    
    Efx2 = Efx2.replace(st[j], sf_j)
        
    Edfxdx = Edfxdx.replace(st[j], sf_j)
# _end_for_

# Finale message.
print("Done!")

## Put all the $E_{sde}$ terms together.

These is given by the expression:

$\int_a^b E_{sde}(t)dt = \frac{1}{2} \sum_{i=1}^{D} \frac{1}{\sigma_{i}^2} \int_a^b \bigg( E[f_{i}^2]_{q_t} -2 E[f_{i}]_{q_t} \dot{m_i}(t) + \dot{m_i}^2(t) + \frac{(\dot{s_i}(t) - \sigma_{i}^2)^2}{4 s_{i}(t)} + (\sigma_{i}^2 - \dot{s_i}(t)) E\big[\frac{\partial f_i}{\partial x_i}\big]_{q_t}\bigg) dt$

where $E[\cdot]_{q_t}$ is the expected value with respect to the Gaussian process.

In essence the following cell computes the integral of each of these terms and adds them all together.

In [ ]:
# Stores the Integral of Esde_i.
integral_Esde = []

# Get the expression for each dimension.
for i in range(dim_D):
        
    # Display info.
    print(f"Estimating the Integral of Esde_{i} ...")
    
    # Part: 1
    ig_01 = sym.integrate(Efx2[i], (t, ti, tj))
    print(" >> ig_01 is finished ")
    
    # Part: 2
    ig_02 =  sym.integrate((Efx[i] * dMt[i]), (t, ti, tj))
    print(" >> ig_02 is finished ")
    
    # Part: 3
    ig_03 = sym.integrate(dMt[i]**2, (t, ti, tj))
    print(" >> ig_03 is finished ")
        
    # Part: 4
    ig_04_01 = sym.integrate((dSt[i]**2)/Sfun[i], (t, ti, tj))
    print(" >> ig_04_01 is finished ")
    
    ig_04_02 = sym.integrate((dSt[i]*sig[i])/Sfun[i], (t, ti, tj))
    print(" >> ig_04_02 is finished ")
    
    ig_04_03 = sym.integrate((sig[i]**2)/Sfun[i], (t, ti, tj))
    print(" >> ig_04_03 is finished ")
    
    # Part: 5
    ig_05 = sym.integrate((sig[i] - dSt[i])*Edfxdx[i], (t, ti, tj))
    print(" >> ig_05 is finished ")

    # Final expression of the Integral ...
    integral_Esde.append(ig_01 - (2.0 * ig_02) + ig_03 + (0.25 * ig_04_01) - (0.5 * ig_04_02) + (0.25 * ig_04_03) + ig_05)
    print("  ")
    
# _end_for_

print("Done!")

In [ ]:
# Print the integral functions.
for n, En in enumerate(integral_Esde_dk):
    print(n, ":")
    sym.pprint(En)

## Compute the gradients.

In [ ]:
# Define the gradient lists.
gradMp = []
gradSp = []

# Process each dimension separately.
for i in range(dim_D):
    
    # Display info.
    print(f"Calculating gradients for dimension: {i} ...")
    
    # Reset the lists for each dimension.
    gradMi = []
    gradSi = []
    
    # Derivatives with respect to the mean parameters.
    for dim_k in Mpar:
        for mk in dim_k:
            gradMi.append(integral_Esde_dk[i].diff(mk))
        # _end_for_
    # _end_for_
    
    # Derivatives with respect to the variance parameters.
    for dim_k in Spar:
        for sk in dim_k:
            gradSi.append(integral_Esde_dk[i].diff(sk))
        # _end_for_
    # _end_for_
    
    # Update the lists.
    gradMp.append(gradMi)
    gradSp.append(gradSi)
    
# _end_for_

print("Done!")

## Generating lambda functions.

In [ ]:
# Lists to store the results.
func_ig_Esde = []
grad_ig_Esde_mt = []
grad_ig_Esde_st = []

for i in range(dim_D):
    
    # Display information.
    print(f"Generating Lambda functions: {i} ... ")
    
    func_ig_Esde.append(sym.lambdify([*flatten([ti, tj, t_h[1:3], t_c[1:2], Mpar, Spar, sig, p])],
                                     integral_Esde_dk[i], modules=["scipy", "numpy"], cse=True))
    
    grad_ig_Esde_mt.append(sym.lambdify([*flatten([ti, tj, t_h[1:3], t_c[1:2], Mpar, Spar, sig, p])],
                                        gradMp[i], modules=["scipy", "numpy"], cse=True))

    grad_ig_Esde_st.append(sym.lambdify([*flatten([ti, tj, t_h[1:3], t_c[1:2], Mpar, Spar, sig, p])],
                                        gradSp[i], modules=["scipy", "numpy"], cse=True))
# _end_for_

# Final message.
print("Done!")

## Save the functions to files.

In [ ]:
# Where to save the functions (parent directory).
# NOTE: We save the energy and the gradient (lambdified) functions in separate directories
# to improve the readability of the code. In practise they can be anywhere.
parent_path = Path("path/to/save/the/equations/of/MeanFieldVarGP/src/dynamical_systems")

for i in range(dim_D):
    
    with open(Path(parent_path / f"integrals/integral_DW_Esde_{i}.sym"), "wb") as sym_file:
        
        dill.dump(func_ig_Esde[i], sym_file)
        
    # _end_with_
    
    with open(Path(parent_path / f"integrals/integral_dDW_Esde_dM{i}.sym"), "wb") as sym_file:
        
        dill.dump(grad_ig_Esde_mt[i], sym_file)
        
    # _end_with_
    
    with open(Path(parent_path / f"integrals/integral_dDW_Esde_dS{i}.sym"), "wb") as sym_file:
        
        dill.dump(grad_ig_Esde_st[i], sym_file)
        
    # _end_with_
    
# _end_for_

print("Done!")

## End of notebook